# List comprehension vs. filter #

In this notebook, I explore the performance of Python list comprehension vs. built-in `filter`.  I discuss why we see the performance difference for a variety of test cases.

## Problem ##

There have been some arguments and confusion about this simple question:  Which is faster for filtering a list by a predicate, [list comprehension expression](https://docs.python.org/2/reference/expressions.html#grammar-token-list_comprehension "Formal definition of Python list comprehension") ("listcomp"), or the built-in [`filter`](https://docs.python.org/2/library/functions.html#filter "Python Library Reference: filter") function?


The answer depends on the nature of the predicate.  In general, there can be three cases:

1.  If the predicate can be incorporated into the listcomp as an efficient expression, but has to be implemented as a Python callable as passed to `filter`, listcomp is usually faster.
2.  If the predicate is simply "to be evaluated to True as a boolean", `filter` is usually the winner.
3.  If the predicate must be evaluated by calling some Python callable no matter in listcomp or `filter` call, the latter is usually faster.

I will demonstrate each case by running some examlple code in `IPython`.

*NOTE:*

In the following post, I will speak often about "Python callable call".  This sounds repetitious, but it's my way of avoid saying "Python function call".  Not all Python callables are functions.

## Explorations ##

I first run some setup code and create some fake input for our test.  I create an input list of 500000 random ASCII characters.

In [1]:
import string
import random
words = [random.choice(string.ascii_uppercase + string.ascii_lowercase)
         for _ in xrange(500000)]

### Case 1: Simple predicate expression in listcomp, unavoidable callable call in `filter` ###

In this case, we want to trim the input list so that all vowel letters are filtered out.  The natural thing to do is to create a `set` of vowels and use set membership as predicate.

In [2]:
vowels = set("aeiouAEIOU")

To compare the speed, there is a handy `IPython` "macro", `%timeit`.  I test listcomp and `filter` call as follows:

In [3]:
%timeit [w for w in words if w not in vowels]

10 loops, best of 3: 28.8 ms per loop


In [4]:
%timeit filter(lambda w: w not in vowels, words)

10 loops, best of 3: 56.9 ms per loop


As can be seen, the latter (`filter` call) is much slower.

Why is that?  Either one seems to be just a loop in which the predicate is tested upon each element, one by one.  The predicate is set membership
```python
w not in vowels
```
which is $\mathcal{O}(1)$ in average case and $\mathcal{O}(n)$ in the worst case (when all hash keys collide).  Therefore, both should be $\mathcal{O}(n)$ in average case (where $n$ is the input size).


However, in the `filter` approach, the predicate is tested by calling the `lambda` expression, a Python callable object written in pure Python.  Even if the `lambda` body is one simple expression, its evaluation imposes some overhead:

* Binding local names to objects (remember: Python "variables" are better recognized as names, and "variable assignment" is name binding);
* Creating stack frame, which is a very "rich" object (this is how we get an informative traceback when an exception goes uncaught).

Using the C API, this can done by calling the C function [`PyObject_Call`](https://docs.python.org/2/c-api/object.html#c.PyObject_Call), which is what happens when a Python callback is passed to a built-in.

On the other hand, during the listcomp expression, the overhead of Python callable call may be avoided.  Such is the case when the sub-expression
```
"if" ...
```
in the listcomp expression
```
"[" ... "for" ... "in" ... ["if" ...] "]"
```
does not explicitly call Python callables.  The `in` operator, with arbitrary object and set as left and right operands respectively, still invokes the [`__contains__` method](https://docs.python.org/2/reference/datamodel.html#object.__contains__), but all this is done by internal C code calling function [`set_direct_contains`](https://github.com/python/cpython/blob/2.7/Objects/setobject.c#L1885), and this is not wrapped in a Python callable call (with the associated overhead).

The reason why `filter` is faster, in this case, is that Python callable call is so heavy that it becomes the bottleneck.  Avoiding calling into it saves much time.

An extreme variant of this case is:

In [5]:
%timeit [w for w in words]

100 loops, best of 3: 13.2 ms per loop


In [6]:
%timeit filter(lambda w: True, words)

10 loops, best of 3: 37 ms per loop


### Case 2: Predicate is simply `bool()`, or "Is this object evaluates to `True` in a boolean context?" ###

In this case, I simply want to filter out all empty strings from the list.  I have to create a new test case with some empty strings:

In [7]:
words = [random.choice([""] + list(string.ascii_lowercase))
         for _ in xrange(500000)]

So that I can test as follows:

In [8]:
%timeit [w for w in words if w]

10 loops, best of 3: 20.2 ms per loop


In [9]:
%timeit filter(None, words)

100 loops, best of 3: 5.74 ms per loop


Now `filter` is the winner! What happened?

You may have guessed it.  In this case, even `filter` doesn't call any Python callable object.  Indeed, this is a special case of `filter`'s [internal implementation](https://github.com/python/cpython/blob/2.7/Python/bltinmodule.c#L244 "Python's internal implementation of filter").


Notice that I passed `None` as `filter`'s first argument, which, according to `filter`'s docstring, means simply testing the boolean truth value of the items:

In [10]:
print filter.__doc__

filter(function or None, sequence) -> list, tuple, or string

Return those items of sequence for which function(item) is true.  If
function is None, return the items that are true.  If sequence is a tuple
or string, return the same type, else return a list.


Internally, this is done by a [special case check](https://github.com/python/cpython/blob/2.7/Python/bltinmodule.c#L303 "Implementation detail of filter") in `filter` (comments are mine):

```C
        /* "func" is the first argument of filter()
           "item" is the item just retrieved from the sequence, under truth value check */
        /* If "func" is Python object "bool" or "None" */
        if (func == (PyObject *)&PyBool_Type || func == Py_None) {
            /* Directly check the item's truth value.  "ok" is a C int. */
            ok = PyObject_IsTrue(item);
        }
        else {
            /* Equivalent to Python code: "good = func(item)" */
            PyObject *good;
            PyTuple_SET_ITEM(arg, 0, item);
            /* Equivalent of expensive Python call.  "good" is PyObject* */
            good = PyObject_Call(func, arg, NULL);
            PyTuple_SET_ITEM(arg, 0, NULL);
            /* Exception in last call */
            if (good == NULL) {
                Py_DECREF(item);
                goto Fail_result_it;
            }
            /* Check truth value of last call's return value */
            ok = PyObject_IsTrue(good);
            Py_DECREF(good);
        }
        /* The code further proceed to using the value of "ok" to determine whether to select
           the item for inclusion in the resulting list. */
```

So, when calling `filter(None, ...)` or `filter(bool, ...)`, the entire code path into `PyObject_Call` on each item is avoided.  Instead, the item is tested by [`PyObject_IsTrue`](https://docs.python.org/2/c-api/object.html#c.PyObject_IsTrue), which for most built-in types is simply the retrieval of a number in the object's header.

That explains why `filter` is fast in this case.  But what about listcomp being slow?

This is because the listcomp, as a Python expression, has larger evaluation overhead in the interpreter, compared to a call to `filter` which is not even a Python function.  Explaining how the interpreter goes on to execute Python bytecode is out of the scope of this post, but I can assure you it's heavy.  The "heaviness" can be glimpsed by looking at the fact that after the evaluation of listcomp expression
```python
[w for w in words if w]
```
the intermediate variable `w` retains its last iteration's value:

In [11]:
[w for w in words if w]
w

'h'

In [12]:
w == words[-1]

True

Indeed, during the listcomp evaluation, all the intermediate steps are done using a **Python** variable `w`, including getting each successive item off the iterator `words`, and appending a selected item to the list being constructed.  Throughout this process, `w` must remain accessible to the whole world (interpreter), and the job of maintaining this variable has a cost on the interpreter.

This stands in contrast to the `filter` approach, where the execution flow enters the "black box" of built-in function.  Inside this black box, the intermediate variables is inaccessible to the Python side of the interpreter and they don't have to be managed.  The list creation and appending operations are still done, fundamentally, using the same code (written in C and compiled to machine code) that directly manipulate the list object.  But between each of those operations (such as [`PyIter_Next`](https://docs.python.org/2/c-api/iter.html#c.PyIter_Next) and [`PyList_SET_ITEM`](https://docs.python.org/2/c-api/list.html#c.PyList_SET_ITEM)), much less is happening.

In addition, `filter` tries to some deliberate optimizations:

*  If the input sequence is a list and is a "singleton" with refcount 1, it is `Py_INCREF`'ed and [modified in-place](https://github.com/python/cpython/blob/2.7/Python/bltinmodule.c#L279);
*  When possible, `filter` [pre-allocates the result list](https://github.com/python/cpython/blob/2.7/Python/bltinmodule.c#L285) using the length hint of input sequence;
*  List append operations are [minimized](https://github.com/python/cpython/blob/2.7/Python/bltinmodule.c#L320) and in-place `PyList_SET_ITEM` is used whenever possible on the pre-allocated list.

In contrast, the list comprehension [does not pre-allocate](https://bugs.python.org/issue14126).

### Case 3: The predicate is expensive for either listcomp or `filter` ###

This is the case when the predicate implements some complex logic, by one way or another, using some Python code.  No matter one is using listcomp or filter, the evaluation must call Python callable.  Note that the mere appearance can be deceiving.  For example, again consider the listcomp expression
```python
[w for w in words if w]
```
When elements in `words` implements their own logic of boolean truth, for example, by implementing the [`__nonzero__`](https://docs.python.org/2/reference/datamodel.html#object.__nonzero__) method in Python code, this apparently clean code _will_ call Python method along with all the overhead, even if the implementation is simply
```python
    ...
    def __nonzero__(self):
        return True
```

To demonstrate, first create a callable to be forced in either expressions:

In [13]:
def predicate(char):
    return char not in vowels

And test:

In [14]:
%timeit [w for w in words if predicate(w)]

10 loops, best of 3: 66.4 ms per loop


In [15]:
%timeit filter(predicate, words)

10 loops, best of 3: 52.2 ms per loop


Now `filter` is still faster, which is expected from our experience with the previous case.  But the cost of calling `predicate` far outweighs everything else.  Compared with [Case 1](#Case-1:-Simple-predicate-expression-in-listcomp,-unavoidable-callable-call-in-filter), the listcomp performs much more poorly.

## Conclusion ##

List comprehension vs. `filter`, which is faster?  In general, `filter` should be faster, because it is implemented using pre-compiled C code (that's what "built-in" means) which simplifies the execution by not taking unecessary steps in Python-land.  It also enables more optimizations such as pre-allocation and highly efficient in-place modification of Python list.

But in practice, the answer usually depends on what kind of predicate condition is used in each case.  Whenever pure Python callable must be iteratively called to determine the truth value of each item from the source iterator, the performance sufferes.

*  For listcomp, this happens when the `if ...` clause must call Python callable, but can be avoided when the condition is a simple expression which has efficient built-in implementation.
*  For `filter`, this can be avoided when the first argument, the "filter function", is `bool` or `None`.  For any other case, calling a callable seems unavoidable.  And if this callable is implemented in Python, even if it's a simple lambda wrapper around a built-in operation, performance suffers.